In [0]:
# Mount ADLS Gen2
# Required each time cluster is restarted

tiers = ['bronze','silver','gold']
adls_paths = {tier: f"abfss://{tier}@marvinwongstorage.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths['bronze']
silver_adls = adls_paths['silver']
gold_adls = adls_paths['gold']

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[]

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
# Construct API URL with start and end dates
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")


Wrote 306061 bytes.
Data successfully saved to abfss://bronze@marvinwongstorage.dfs.core.windows.net//2025-08-30_earthquake_data.json


In [0]:
data[0]

{'type': 'Feature',
 'properties': {'mag': 0.69,
  'place': '8 km WNW of Cobb, CA',
  'time': 1756598036030,
  'updated': 1756598133772,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75231007',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75231007&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 7,
  'net': 'nc',
  'code': '75231007',
  'ids': ',nc75231007,',
  'sources': ',nc,',
  'types': ',nearby-cities,origin,phase-data,',
  'nst': 7,
  'dmin': 0.01577,
  'rms': 0,
  'gap': 124,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 0.7 - 8 km WNW of Cobb, CA'},
 'geometry': {'type': 'Point',
  'coordinates': [-122.810997009277, 38.839168548584, 1.75999999046326]},
 'id': 'nc75231007'}

In [0]:
output_data = {
    'start_date': start_date.isoformat(),
    'end_date': end_date.isoformat(),
    'bronze_adls': bronze_adls,
    'silver_adls': silver_adls,
    'gold_adls': gold_adls
}

# Return dictionary
dbutils.jobs.taskValues.set(key = "bronze_output", value = output_data)